# Topic Models

In [ ]:
es = pd.read_csv('data/ES.csv')
es = es[es.Time == '15:00'][['Date','Time','Close']]
es.Date = pd.to_datetime(es.Date +' ' +'15:45')
es.drop('Time',axis=1,inplace=True)
es.set_index('Date', inplace=True)

date_range = es['2011-12-30':'2020-5-31'].index

es['Post'] = np.roll(es['Close'].pct_change(),-1)

plt.plot(es['Close'])

import statsmodels.api as sm

X_train = nmf_features[:'2016-12-31']
X_test = nmf_features['2016-12-31':]

y = 1*(y >0)

y_train = y[:'2016-12-31']
y_test = y['2016-12-31':]

from sklearn.metrics import f1_score, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

scores = []
f1scores = []
roc = []
for i in range(2,16):
    clf = RandomForestClassifier(max_depth=i, random_state=0)
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))
    f1scores.append(f1_score(clf.predict(X_test),y_test))
    roc.append(roc_auc_score(clf.predict(X_test),y_test))
#clf.score(X_train, y_train)

clf = RandomForestClassifier(max_depth=8, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

clf.feature_importances_

plt.plot(range(2,16),roc)

# Sentiment

In [ ]:
X_train = user_features[:'2016-12-31']
X_test = user_features['2016-12-31':]

y_vader = y[~user_features['PipCzar'].isnull()]
X_vader = user_features['PipCzar'][~user_features['PipCzar'].isnull()]

X_vader_train = X_vader[:'2016-12-31']
X_vader_test = X_vader['2016-12-31':]

y_vader_train = y_vader[:'2016-12-31']
y_vader_test = y_vader['2016-12-31':]

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(np.array(X_vader_train).reshape(-1, 1), y_vader_train)
clf.score(np.array(X_vader_test).reshape(-1, 1), y_vader_test)

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)




X_train = user_features[:'2016-12-31']
X_test = user_features['2016-12-31':]

y = es['Post'].loc[sentiment_features[['daily_average']].index]
y = 1*(y >0)

y_train = y[:'2016-12-31']
y_test = y['2016-12-31':]

models = []

for handle in user_features.columns:
    mask = ~user_features[handle].isnull()
    if mask.sum() >= 300:
        X_handle = np.array(user_features[handle][mask]).reshape(-1,1)
        y_handle = y[mask] 
        clf = LogisticRegression(random_state=0).fit(X_handle, y_handle)
        models.append((handle, float(clf.coef_), clf.score(X_handle, y_handle) ))
 

xxx = pd.DataFrame(models, columns=['handle','coef','score']).sort_values(by=['score'])[-100:]